In [ ]:
from __future__ import print_function
import os
from netCDF4 import Dataset
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from wrf import to_np, getvar, smooth2d, get_cartopy, cartopy_xlim, cartopy_ylim, latlon_coords, disable_xarray
%matplotlib notebook

In [ ]:
# Set up the base output directory, the time we want to look at, and the member number we want to plot
# Note, for your stuff you may want to modify this to loop through various members, calculate averages, etc.
# Please experiment!
outputbasedir = '/depot/dawson29/data/082416_tornado_outbreak/simulations/WRFDART/15km1hr/output'
timestamp = '2016082420'
member = 1

In [ ]:
# Construct the full path to the file
outputdir = os.path.join(outputbasedir, timestamp)
outputdir = os.path.join(outputdir, "PRIORS")
filename = 'prior_d01.{:04d}'.format(member)
filepath = os.path.join(outputdir, filename)

# Make sure the path exists. If this comes out False, something's wrong.
print(os.path.exists(filepath))

In [ ]:
# Open the NetCDF file
ncfile = Dataset(filepath)

# Calculate and retrieve CAPE
cape_package = getvar(ncfile, 'cape2d')
cape = cape_package[0]

In [ ]:
# Get the latitude and longitude points
lats, lons = latlon_coords(cape)
# Get the cartopy mapping object
cart_proj = get_cartopy(cape)

In [ ]:
# Create a figure
fig = plt.figure(figsize=(12,9))
# Set the GeoAxes to the projection used by WRF
ax = plt.axes(projection=cart_proj)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m', edgecolor='k', facecolor='none',
                             name='admin_1_states_provinces_shp')
ax.add_feature(states, linewidth=.5)
ax.coastlines('50m', linewidth=0.8)

# Make the contour outlines and filled contours for the smoothed sea level pressure.
plt.contour(to_np(lons), to_np(lats), to_np(cape), 10, colors="black",
            transform=crs.PlateCarree())
plt.contourf(to_np(lons), to_np(lats), to_np(cape), 10, transform=crs.PlateCarree(),
             cmap=get_cmap("jet"))

# Add a color bar
plt.colorbar(ax=ax, shrink=.62)

# Set the map limits.  Not really necessary, but used for demonstration.
ax.set_xlim(0,1900000)
ax.set_ylim(-750000,1100000)

# Add the gridlines
ax.gridlines(color="black", linestyle="dotted")

plt.title("CAPE J/kg")